In [139]:
import os
import openai
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# %%time
# import os
# ! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
# ! pip install -qq -U transformers InstructorEmbedding sentence_transformers
# ! pip install -qq -U accelerate bitsandbytes xformers einops

In [3]:
# pip install transformers

In [4]:
# pip install sentence-transformers

In [5]:
# pip install pyopenssl --upgrade

In [6]:
# pip install --no-cache fastapi

In [5]:
# !pip install typing-inspect 

In [3]:
# pip install typing_extensions==4.9.0

In [9]:
# !pip install typing_extensions  

In [10]:
# pip install unstructured

In [11]:
# pip install langchain --upgrade

In [12]:
# !pip install langchain==0.0.018
# !pip show langchain

In [13]:
# !pip install pydantic 

In [6]:
import langchain

In [7]:
import warnings
warnings.filterwarnings("ignore")
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.vectorstores import Chroma
# from langchain.vectorstores.chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
import torch

import langchain
print(langchain.__version__)

2024-03-04 05:52:05.422644: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-04 05:52:05.422678: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


0.1.9


In [8]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("Lean/")
transcript = loader.load()

In [17]:
# with open('Beel_0-285593.json', 'r') as file:
#     text_data = file.read()
    

In [9]:
# Now you have the text data as a string, and you can split it using the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
split_texts = text_splitter.split_documents(transcript)
# split_texts = text_splitter.create_documents(transcript)


In [10]:
len(split_texts)

22

In [20]:
###Embedding and convert chunks to Vectordb using chroma##

In [21]:
# !pip uninstall transformers -y
# !pip install sentence-transformers==2.2.2

In [22]:
%%time

# ! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
# ! pip install -qq -U  InstructorEmbedding sentence_transformers
# ! pip install -qq -U accelerate bitsandbytes xformers einops

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.44 µs


In [23]:
# %%time
### this takes ~35 min to run
from langchain.embeddings import HuggingFaceInstructEmbeddings

persist_directory = 'Lean_pdf'

### download embeddings model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

### create embeddings and DB
vectordb = Chroma.from_documents(documents=split_texts,
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory
                                )



### persist Chroma database
vectordb.persist()



load INSTRUCTOR_Transformer
max_seq_length  512


In [24]:
# !pip install transformers --upgrade

In [11]:
# %%time
from langchain.embeddings import HuggingFaceInstructEmbeddings

load_directory = 'Lean_pdf'

### download embeddings model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl"
                                                     )

vectordb = Chroma(persist_directory=load_directory, embedding_function=instructor_embeddings)



load INSTRUCTOR_Transformer
max_seq_length  512


In [12]:
## use falcon LLM model or any other model##

In [13]:
# !pip install git+https://github.com/huggingface/transformers

In [14]:
# !pip uninstall torch transformers -y
# !pip install torch transformers

In [15]:
# pip install transformers accelerate bitsandbytes

In [16]:
# pip install --upgrade transformers

In [17]:
import torch
# tensor = torch.empty(shape)
# device = torch.device("cpu")
from peft import PeftModel 
from transformers import AutoModelForCausalLM, AutoTokenizer,LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
# from transformers.models.bert.modeling_bert import BertModel,BertForMaskedLM
# quantization_config = BitsAndBytesConfig(llm_int8_threshold=200.0)
tokenizer = AutoTokenizer.from_pretrained('daryl149/llama-2-13b-chat-hf')
 # daryl149/llama-2-13b-chat-hf       
model = AutoModelForCausalLM.from_pretrained("daryl149/llama-2-13b-chat-hf",
                                             # load_in_4bit=True,
                                             device_map='auto',
                                             torch_dtype=torch.bfloat16,
                                             low_cpu_mem_usage=True,
                                             # trust_remote_code=True,
                                             # quantization_config=quantization_config,
                                            )
max_len = 4096
task = "text-generation"
T = 0.1

/home/sagemaker-user/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
## Pass llm model into the huggingface Pipeline###

In [19]:
pipe = pipeline(
    task=task,
    model=model, 
    tokenizer=tokenizer, 
    max_length=max_len,
    temperature=T,
    top_p=0.95,
    repetition_penalty=1.1
)
llm = HuggingFacePipeline(pipeline=pipe)

In [20]:
## pass full pipeline into the QA chain####


In [25]:
from langchain import PromptTemplate, LLMChain
# _template = """Given the following conversation and a follow up question, rephrase the follow up question to be a 
# standalone question without changing the content in given question.

# Chat History:
# {chat_history}
# Follow Up Input: {question}
# Standalone question:"""
# condense_question_prompt_template = PromptTemplate.from_template(_template)

# prompt_template = """You are helpful information giving QA System and make sure you don't answer anything 
# not related to following context. You are always provide useful information & details available in the given context. Use the following pieces of context to answer the question at the end. 
# If you don't know the answer, just say that you don't know, don't try to make up an answer. 

# {context}

# Question: {question}
# Helpful Answer:"""
# Define prompt template
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use two sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

qa_prompt = PromptTemplate.from_template(template)

# qa_prompt = PromptTemplate(
#     template=prompt_template, input_variables=["context", "question"]
# )

In [26]:
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from functools import lru_cache

# Initialize a ConversationalMemory instance
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
question_generator = LLMChain(llm=llm, prompt=qa_prompt, memory=memory)
doc_chain = load_qa_chain(llm, chain_type="stuff")
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    # return_source_documents=True,
    verbose=True,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    memory=memory
)

# # Define a function to cache retriever results
# @lru_cache(maxsize=None)  # None indicates unlimited cache size
# def cached_retriever_result(question):
#     return qa_chain({'question': question, 'chat_history': chat_history})

# # Rest of your code
# chat_history = []
# while True:
#     question = input("You: ")  # Get user input

#     # Query the cached retriever function
#     result = cached_retriever_result(question)

#     response = result['answer']
#     chat_history.append((question, response))

#     print("AI:", response)



In [27]:
from datasets import Dataset

questions = ["what is Six Sigma?"]
ground_truths = ["Six Sigma is a process improvement methodology developed at Motorola in the1980’s to reduce defects in its processes. Its goal was to achieve a level ofperformance equal to a defect rate of 3.4 defects per million opportunities – this is avirtually defect free environment i.e. Six Sigma performance"]
answers = []
contexts = []

# Inference
for question in questions:
    answer = qa_chain.invoke(question)
    answer = answer.get('answer')
    context_list = [docs.page_content for docs in retriever.get_relevant_documents(question)]
    
    answers.append(answer)
    contexts.append(context_list)

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}
print(data)
# # Convert dict to dataset
# dataset = Dataset.from_dict(data)



> Entering new ConversationalRetrievalChain chain...

> Finished chain.
{'question': ['what is Six Sigma?'], 'answer': [' Six Sigma is a methodology for improving the efficiency and effectiveness of business processes. It uses statistical tools and techniques to identify and eliminate defects, variations, and waste in order to achieve near-perfect quality levels.'], 'contexts': [['Institute for Innovation\nand Improvement\nLean Six Sigma: \nsome basic concepts', 'Institute for Innovation\nand Improvement\nLean Six Sigma: \nsome basic concepts', 'Institute for Innovation\nand Improvement\nLean Six Sigma: \nsome basic concepts']], 'ground_truth': ['Six Sigma is a process improvement methodology developed at Motorola in the1980’s to reduce defects in its processes. Its goal was to achieve a level ofperformance equal to a defect rate of 3.4 defects per million opportunities – this is avirtually defect free environment i.e. Six Sigma performance']}


In [108]:
# print(contexts)

In [279]:
from datasets import Dataset

questions = ["What is Six Sigma?",
             "What is Six Sigma?"]
ground_truths = ["Six Sigma is a process improvement methodology developed at Motorola in the1980’s to reduce defects in its processes. Its goal was to achieve a level ofperformance equal to a defect rate of 3.4 defects per million opportunities – this is avirtually defect free environment i.e. Six Sigma performance",
                "Innovation\nand Improvement\nLean Six Sigma: Six Sigma focuses primarily on reducing variation, whilst Lean focuses on improving flow in the value"]

answers = ["Six Sigma is a methodology for improving the efficiency and effectiveness of business processes. It uses statistical tools and techniques to identify and eliminate defects, variations, and waste in order to achieve near-perfect quality levels.",
           "Lean techniques, such as value stream mapping, have been used in the NHS since the mid 1990s, largely through episodic Kaizen events or by combining Lean tools withother improvement approaches. "]
contexts = [["'Six Sigma is a methodology efficiency developed at Motorola in .Institute for Innovation\nand Improvement\nLean Six Sigma: \nsome basic concepts', 'Institute for Innovation\nand Improvement\nLean Six Sigma: \nsome basic concepts', 'Institute for Innovation\nand Improvement\nLean Six Sigma: \nsome basic concepts'"],
            ["Institute Lean Six Sigma,Six Sigma focuses primarily"]]

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth":ground_truths
}
# print(data)
# # Convert dict to dataset
dataset = Dataset.from_dict(data)

In [280]:
# dataset = Dataset.from_dict(data)


In [281]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 2
})

In [282]:
# pip install datasets

In [283]:
# pip install openai

In [284]:
from ragas import evaluate
from ragas.metrics import (
    # faithfulness,
    # answer_relevancy,
    context_recall,
    # context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        # context_precision,
        context_recall,
        # faithfulness,
        # answer_relevancy,
        
    ]
)

df = result.to_pandas()

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [285]:
df

,question,answer,contexts,ground_truth,context_recall
0,What is Six Sigma?,Six Sigma is a methodology for improving the e...,['Six Sigma is a methodology efficiency develo...,Six Sigma is a process improvement methodology...,1.0
1,What is Six Sigma?,"Lean techniques, such as value stream mapping,...","[Institute Lean Six Sigma,Six Sigma focuses pr...",Innovation\nand Improvement\nLean Six Sigma: S...,1.0


In [247]:
# pip install 

In [ ]:


def final_text(text, width=160000):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text


In [ ]:
def final_llm_response(llm_final_response):
    print(final_text(llm_final_response['result']))

def final_answer(question):
    llm_final_response = qa_chain(question)
    answer = final_llm_response(llm_final_response)
    return answer

In [ ]:
# %%time
# question = "A farm house uses atleast 800 kg of special food daily. The special food is a mixture of corn and soyabean with the following compositions"
# final_answer(question)

In [ ]:
%%time
question = "what is six sigma?"
final_answer(question)